## Notebook used for data augmentation
#### Functions in data_augmentation.py

In [1]:
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import numpy as np

from tf_aerial_images import *

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from data_augmentation import *

In [3]:
data_dir = "./data/training/"
train_data_filename = data_dir + "images/"
train_labels_filename = data_dir + "groundtruth/"

In [7]:
segmproc = Image.fromarray(np.where(np.array(segmentation) > 150, 255, 0).astype(np.uint8))

In [ ]:
# Random rotations
rotations_images_dir = train_data_filename + "random_rotations/"
rotations_labels_dir = train_labels_filename + "random_rotations/"

if not os.path.exists(rotations_images_dir):
    os.makedirs(rotations_images_dir)

if not os.path.exists(rotations_labels_dir):
    os.makedirs(rotations_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Rotate the image and its segmentation by a random angle
    img_rot, segm_rot = rotate(img, segm)
    img_rot.save(rotations_images_dir + im[:-4] + "_rot.png")
    segm_rot.save(rotations_labels_dir + im[:-4] + "_rot.png")

In [7]:
# Set rotations
rotations = [90, 180, 270]
set_rotations_images_dir = train_data_filename + "set_rotations/"
set_rotations_labels_dir = train_labels_filename + "set_rotations/"

if not os.path.exists(set_rotations_images_dir):
    os.makedirs(set_rotations_images_dir)

if not os.path.exists(set_rotations_labels_dir):
    os.makedirs(set_rotations_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Rotate the image and its segmentation by a set of angles
    for angle in rotations:
        img_rot, segm_rot = rotate(img, segm, angle)
        img_rot.save(set_rotations_images_dir + im[:-4] + "_rot_" + str(angle) + ".png")
        segm_rot.save(set_rotations_labels_dir + im[:-4] + "_rot_" + str(angle) + ".png")

In [8]:
# Horizontal and vertical flips
flips_images_dir = train_data_filename + "flips/"
flips_labels_dir = train_labels_filename + "flips/"

if not os.path.exists(flips_images_dir):
    os.makedirs(flips_images_dir)

if not os.path.exists(flips_labels_dir):
    os.makedirs(flips_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Flip the image and its segmentation horizontally and vertically
    img_flip_h, segm_flip_h = flip(img, segm, horizontal=True)
    img_flip_v, segm_flip_v = flip(img, segm, horizontal=False)
    img_flip_h.save(flips_images_dir + im[:-4] + "_flip_h.png")
    segm_flip_h.save(flips_labels_dir + im[:-4] + "_flip_h.png")
    img_flip_v.save(flips_images_dir + im[:-4] + "_flip_v.png")
    segm_flip_v.save(flips_labels_dir + im[:-4] + "_flip_v.png")

In [9]:
# Blurring
blurs_images_dir = train_data_filename + "blurs/"
blurs_labels_dir = train_labels_filename + "blurs/"

if not os.path.exists(blurs_images_dir):
    os.makedirs(blurs_images_dir)

if not os.path.exists(blurs_labels_dir):
    os.makedirs(blurs_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Apply a random gaussian blur to the image
    img_blur, segm_blur = blur(img, segm)
    img_blur.save(blurs_images_dir + im[:-4] + "_blur.png")
    segm_blur.save(blurs_labels_dir + im[:-4] + "_blur.png")

In [10]:
# Jittering
jitters_images_dir = train_data_filename + "jitters/"
jitters_labels_dir = train_labels_filename + "jitters/"

if not os.path.exists(jitters_images_dir):
    os.makedirs(jitters_images_dir)

if not os.path.exists(jitters_labels_dir):
    os.makedirs(jitters_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Apply a random jitter to the image
    img_jitter, segm_jitter = jitter(img, segm)
    img_jitter.save(jitters_images_dir + im[:-4] + "_jitter.png")
    segm_jitter.save(jitters_labels_dir + im[:-4] + "_jitter.png")

In [11]:
# Add gaussian noise
noises_images_dir = train_data_filename + "noises/"
noises_labels_dir = train_labels_filename + "noises/"

if not os.path.exists(noises_images_dir):
    os.makedirs(noises_images_dir)

if not os.path.exists(noises_labels_dir):
    os.makedirs(noises_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Add random gaussian noise to the image
    stdev = random.randint(1, 10)
    img_noise, segm_noise = gaussian_noise(img, segm, stdev)
    img_noise.save(noises_images_dir + im[:-4] + "_noise.png")
    segm_noise.save(noises_labels_dir + im[:-4] + "_noise.png")

In [12]:
# Random crop and resize
cropresize_images_dir = train_data_filename + "cropresize/"
cropresize_labels_dir = train_labels_filename + "cropresize/"

if not os.path.exists(cropresize_images_dir):
    os.makedirs(cropresize_images_dir)

if not os.path.exists(cropresize_labels_dir):
    os.makedirs(cropresize_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Randomly crop and resize the image
    img_crop, segm_crop = random_crop(img, segm)
    img_crop.save(cropresize_images_dir + im[:-4] + "_cropresize.png")
    segm_crop.save(cropresize_labels_dir + im[:-4] + "_cropresize.png")

In [8]:
# Gray scale
gray_images_dir = train_data_filename + "gray/"
gray_labels_dir = train_labels_filename + "gray/"

if not os.path.exists(gray_images_dir):
    os.makedirs(gray_images_dir)

if not os.path.exists(gray_labels_dir):
    os.makedirs(gray_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Convert the image to gray scale
    img_gray, segm_gray = grayscale(img, segm)
    img_gray.save(gray_images_dir + im[:-4] + "_gray.png")
    segm_gray.save(gray_labels_dir + im[:-4] + "_gray.png")

In [14]:
# Invert colors
invert_images_dir = train_data_filename + "invert/"
invert_labels_dir = train_labels_filename + "invert/"

if not os.path.exists(invert_images_dir):
    os.makedirs(invert_images_dir)

if not os.path.exists(invert_labels_dir):
    os.makedirs(invert_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Invert the colors of the image
    img_inv, segm_inv = invert_colors(img, segm)
    img_inv.save(invert_images_dir + im[:-4] + "_invert.png")
    segm_inv.save(invert_labels_dir + im[:-4] + "_invert.png")

In [12]:
# All transformations
all_images_dir = train_data_filename + "random_all_augmentations/"
all_labels_dir = train_labels_filename + "random_all_augmentations/"

if not os.path.exists(all_images_dir):
    os.makedirs(all_images_dir)

if not os.path.exists(all_labels_dir):
    os.makedirs(all_labels_dir)

# Loop over all original images in the training set
for im in os.listdir(train_data_filename):
    # Ignore files that are not images
    if not im.endswith(".png"):
        continue

    # Open the image and its segmentation
    img = Image.open(train_data_filename + im)
    segm = Image.open(train_labels_filename + im)

    # Apply all transformations to the image
    img_all, segm_all = apply_all(img, segm)
    img_all.save(all_images_dir + im[:-4] + "_rand_all.png")
    segm_all.save(all_labels_dir + im[:-4] + "_rand_all.png")